In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("products.txt")
documents = loader.load()

documents[0]


Document(metadata={'source': 'products.txt'}, page_content='Product: iPhone 14\nCategory: Mobile\nPrice: 799\nFeatures: A15 Bionic chip, 128GB storage, Dual camera.\n\nProduct: Samsung Galaxy S23\nCategory: Mobile\nPrice: 749\nFeatures: Snapdragon processor, AMOLED display, Triple camera.\n\nProduct: MacBook Air M2\nCategory: Laptop\nPrice: 1199\nFeatures: Apple M2 chip, 16GB RAM, 512GB SSD.\n\nProduct: Dell XPS 13\nCategory: Laptop\nPrice: 999\nFeatures: Intel i7, 16GB RAM, 1TB SSD.\n')

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

chunks = splitter.split_documents(documents)

len(chunks)


2

In [3]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3"
)


C:\Users\apoliset\AppData\Local\Temp\ipykernel_7596\4053169012.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [4]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)


In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


In [6]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")


C:\Users\apoliset\AppData\Local\Temp\ipykernel_7596\2342230918.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


In [7]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
You are an e-commerce assistant.
Answer the question using ONLY the context below.
If the answer is not present, say "I don't know".

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [9]:
rag_chain.invoke("What is the price of MacBook Air M2?")


'1199'

In [10]:
rag_chain.invoke("Which laptop has 1TB SSD?")


'According to the context, the Dell XPS 13 has a 1TB SSD.'

In [11]:
rag_chain.invoke("Suggest a mobile phone under $800")


"I'd suggest the Samsung Galaxy S23 as it falls under $800 with a price tag of $749."